# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

##  Изучение общей информации

Ход исследования

Данные о заемщиках получаем из файла /datasets/data.csv. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных.

Проверяем данные на ошибки и оцениваем их влияние на исследование. Затем, на этапе предобработки ищем возможность исправить самые критичные ошибки данных.

Таким образом, исследование пройдёт в три этапа:

1. Обзор данных.
2. Предобработка данных.
3. Проверка гипотез.



In [1]:
# импорт библиотеки pandas
import pandas as pd

In [2]:
# чтение файла с данными и сохранение в df
df = pd.read_csv('/datasets/data.csv') 

In [3]:
# получение первых 10 строк таблицы df
df.head(10) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
# получение общей информации о данных в таблице df
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
#проверим величины хранящиеся в таблице
df.describe() 

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

Итак, в таблице двенадцать столбцов. Тип данных в 2 столбцах — float, в 5 столбцах - int, в 5 столбцах - object.

Согласно документации к данным:

* children — количество детей в семье,
* days_employed — общий трудовой стаж в днях,
* dob_years — возраст клиента в годах,
* education — уровень образования клиента,
* education_id — идентификатор уровня образования,
* family_status — семейное положение,
* family_status_id — идентификатор семейного положения,
* gender — пол клиента,
* income_type — тип занятости,
* debt — имел ли задолженность по возврату кредитов,
* total_income — ежемесячный доход,
* purpose — цель получения кредита.

В колоноках видны три нарушения стиля:

* Строчные буквы сочетаются с прописными.
* В колонке 'days_employed' встречаются отрицательные значения.
* В 'total_income'  и 'days_employed' встречаются «нулевые», пропущенные значения, также тип данных не соотвествует действительности. 

В каждой строке таблицы — данные о заемщике,которые рассказывают о нем: его возраст, сколько у него детей, семейное положение и т.п. 
Предварительно можно утверждать, что, данных достаточно для проверки гипотез, но встречаются пропуски в данных.
Чтобы двигаться дальше, нужно устранить проблемы в данных.

##  Предобработка данных

### Обработка пропусков
Сначала считаем, сколько в таблице пропущенных значений. Для этого достаточно двух методов pandas:

In [6]:
# подсчёт пропусков
df.isna().sum() 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Не все пропущенные значения влияют на исследование. Так в 'days_employed' пропуски не важны для вашей работы. Достаточно заменить их явными обозначениями.

Но пропуски в 'total_income' могут помешать исследованию. На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. 
Придётся:
* заполнить и эти пропуски явными обозначениями,
* оценить, насколько они повредят расчётам. 

 Для расчетов нам понадобиться только столбец 'total_income', поэтому и начнем его изменять.

In [7]:
# считаем пропущенные значения методом isna()
df.isna().mean() 

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

Для заполнения пропущенных значений в строке 'total_income', мы посчитаем медианные значения дохода по типам занятости, и заполним ими пропуски.Для замены строковых значений вызывают атрибут loc, он осуществляет логическую индексацию — выбор значений по заданному условию. Логическая индексация позволяет найти в столбце пропуски или другие значения типа str и заменить их.


In [8]:
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform("median"))

In [9]:
# получение общей информации о данных в таблице df
df.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

В таблице были пропуски в показателях стажа и в показателях ежемесячного дохода,  которые были заменены медианным значением дохода.Больше никаких критических пропусков не выявлено, но лучше проверить аномалии для каждого столбца отдельно. 

### Замена типа данных

In [10]:
# воспользовавшись методом 'astype', преобразовали столбец с общим доходом к целочисленному типу данных
df['total_income'] = df['total_income'].astype('int') 

In [11]:
 # получение общей информации о данных в таблице df
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод**

Воспользовавшись методом astype, преобразовали столбец с общим доходом  к целочисленному типу данных.

### Обработка дубликатов

В столбце 'education' посчитаем количество дублей для каждого уровня образования клиента методом value_counts().

In [12]:
#посмотрим количество уникальных значений в столбце 'уровень образования клиента'
df['education'].value_counts().sort_index() 

ВЫСШЕЕ                   274
Высшее                   268
НАЧАЛЬНОЕ                 17
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
Начальное                 15
Неоконченное высшее       47
СРЕДНЕЕ                  772
Среднее                  711
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
высшее                  4718
начальное                250
неоконченное высшее      668
среднее                13750
ученая степень             4
Name: education, dtype: int64

Из-за использования заглавных букв в названиях 'уровня образования клиента' образовались дубликаты значений. Приведем к нижнему регистру  методом str.lower()

In [13]:
 #приведем все названия в столбце 'уровень образования клиента' к нижнему регистру
df['education'] = df['education'].str.lower()

In [14]:
df['education'].duplicated().sum()

21520

In [15]:
#посмотрим количество уникальных значений в столбце 'уровень образования клиента'.
df['education'].value_counts().sort_index() 

высшее                  5260
начальное                282
неоконченное высшее      744
среднее                15233
ученая степень             6
Name: education, dtype: int64

** Вывод**

Удалив дубликаты в столбце 'уровень образования клиента'  мы теперь явно видим зависимость, заемщики со средним образованием в 3 раза чаще берут кредит, по сравнению с заемщиками закончившими ВУЗы. Также можем отметить, что всего чуть больше 1% заемщиков имеют начальное образование, что говорит о общей уровне образованности у клиентов банка.

В столбце 'education' посчитаем количество дублей для каждого 'уровня образования клиента' методом value_counts().

In [16]:
#посмотрим количество уникальных значений в столбце 'уровень образования клиента'
df['education'].value_counts().sort_index() 

высшее                  5260
начальное                282
неоконченное высшее      744
среднее                15233
ученая степень             6
Name: education, dtype: int64

In [17]:
#посмотрим количество уникальных значений в столбце "семейное положение"
df['family_status'].value_counts() 

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [18]:
#приведем все названия в столбце "семейное положение" к нижнему регистру
df['family_status'] = df['family_status'].str.lower() 

In [19]:
#посмотрим количество уникальных значений в столбце "семейное положение"
df['family_status'].value_counts() 

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [20]:
#посмотрим количество уникальных значений в столбце "пол клиента"
df['gender'].value_counts() 

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Значение XNA в столбце 'gender'. Клиент не передал информацию о поле при выборе M и F. Это случайный характер ошибки.

In [21]:
df[df['gender'] == 'XNA'].head(10) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


Из таблицы видно,что заемщику 24 года, работает компаньоном, имеет неоконченное высшее образование и т.п. и мы не можем утверждать, что это Ж или М, то оставляем данное значение без изменений.

In [22]:
#посмотрим количество уникальных значений в столбце 'количество детей в семье'
df['children'].value_counts().sort_index() 

-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

Из данных видем, что имеется 2 ошибки (20,-1).Скорее всего это ошибка, где 20 - это 2,а -1 - это 0.

In [23]:
# 20 заменим на 2,т.к. это ошибка перевода в датасет. Человек подразумевал 2 ребенка
df.loc[df['children'] == 20, 'children'] = 2 

In [24]:
# -1 заменим на 0, т.к. скорей подразумевалось отсутсвие детей
df.loc[df['children'] == -1, 'children'] = 0 

In [25]:
#посмотрим количество уникальных значений в столбце 'количество детей в семье'
df['children'].value_counts().sort_index() 

0    14196
1     4818
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [26]:
 #посмотрим количество уникальных значений в столбце 'возраст клиента в годах'
df['dob_years'].value_counts().sort_index()

0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

В колонке есть значение 0.Заполненим пропущенные значения в строке 'возраст клиента в годах' медианными значениями возраста. Медианый доход предпочли среднему доходу, чтобы исключить влияние на данные запредельных показателей.

In [27]:
df['dob_years'] = df['dob_years'].replace(0, int(df['dob_years'][df['dob_years'] != 0].median()))


In [28]:
df['dob_years'][df['dob_years'] == 0].count()

0

In [29]:
#посмотрим количество уникальных значений в столбце 'возраст клиента в годах'
df['dob_years'].value_counts().sort_index() 

19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    614
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [30]:
#посмотрим количество уникальных значений в столбце 'тип занятости'
df['income_type'].value_counts()  

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [31]:
#посмотрим количество уникальных значений в столбце 'общий трудовой стаж в днях'
df['days_employed'].value_counts() 

-327.685916     1
-1580.622577    1
-4122.460569    1
-2828.237691    1
-2636.090517    1
               ..
-7120.517564    1
-2146.884040    1
-881.454684     1
-794.666350     1
-3382.113891    1
Name: days_employed, Length: 19351, dtype: int64

Отрицательное  количество дней стажа в днях необходимо перевести в положительные.

In [32]:
#используем функцию abs() для отрицательных значений, которая возвращает модуль от числа
df['days_employed'] = abs(df['days_employed']) 

In [33]:
df['days_employed'].value_counts()

142.276217       1
1849.622944      1
886.253127       1
2539.534295      1
390574.985524    1
                ..
1394.302246      1
2325.720832      1
4086.407828      1
1259.497032      1
1636.419775      1
Name: days_employed, Length: 19351, dtype: int64

In [34]:
len(df[df['days_employed'].isna()]) 

2174

В колонке 10% значения Nan.Заполним пропущенные значения в строке 'общий трудовой стаж в днях' медианными значениями стажа. Медианый стаж предпочли среднему стажу, чтобы исключить влияние на данные запредельных показателей.Для заполнения пропущенных значений в строке 'общий трудовой стаж', мы посчитаем медианные значения дохода по типам занятости, и заполним ими пропуски. Медианый доход предпочли среднему доходу, чтобы исключить влияние на данные запредельных показателей.Для замены строковых значений вызывают атрибут loc, он осуществляет логическую индексацию — выбор значений по заданному условию. Логическая индексация позволяет найти в столбце пропуски или другие значения типа str и заменить их.


In [35]:
df['days_employed'] = df['days_employed'].fillna(df.groupby('income_type')['days_employed'].transform("median"))

In [36]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


In [37]:
#посмотрим количество уникальных значений в столбце 'цель получения кредита'
df['purpose'].value_counts() 

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

**Вывод**

Аномалиями, которые были решены:

* Из-за использования заглавных букв в названиях 'education' образовались дубликаты значений
* Пропуск в столбце 'gender'.
* Отрицательные и большие положительные значения в столбце days_employed.
* Пропуски в столбцах 'days_employed' и 'total_income'.
* Нулевое значение в столбце 'dob_years'.

Столбец 'purpose' требует выявления общих категорий для одного типа целей кредита.

In [38]:
#найдем количество дубликатов
df.duplicated().sum() 

71

In [39]:
#избавимся от них и восстановим индексы.
df = df.drop_duplicates().reset_index(drop = True) 

In [40]:
#проверим что у нас вышло
df.duplicated().sum() 

0

**Вывод**



* Был найден 71 дубликат от которго мы избавились.


### Лемматизация

In [41]:
# импортируем pymystem3 
from pymystem3 import Mystem 
m = Mystem() 

In [42]:
lemmas = []

In [43]:
purpos = df['purpose'].unique()

In [44]:
for i in purpos:
    lemma = m.lemmatize(i)
    lemmas.append(lemma)

In [45]:
lemmas

[['покупка', ' ', 'жилье', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['дополнительный', ' ', 'образование', '\n'],
 ['сыграть', ' ', 'свадьба', '\n'],
 ['операция', ' ', 'с', ' ', 'жилье', '\n'],
 ['образование', '\n'],
 ['на', ' ', 'проведение', ' ', 'свадьба', '\n'],
 ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'],
 ['покупка', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'],
 ['недвижимость', '\n'],
 ['строительство', ' ', 'недвижимость', '\n'],
 ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'],
 ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'],
 ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['жилье', '\n'],
 ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'],
 ['автомобиль'

 Из полученных лемм видем, что можно выделить 4 категори, а именно: автомобиль, образование, свадьба, недвижимость.

In [46]:

def purposes(purpose):
    lemmas_row = lemma
    for i in lemmas_row:
        if 'авто' in purpose:
            return 'автомобиль'
        if 'недвижим' in purpose or 'жиль' in purpose:
            return  'недвижимость'
        if 'свадьб' in purpose:
            return 'свадьба'
        if 'образов' in purpose:
            return 'образование'
        return 'другое'
df['purposes_list'] = df['purpose'].apply(purposes)   


In [47]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purposes_list
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


In [48]:
df['purposes_list'].value_counts().sort_index()

автомобиль       4306
недвижимость    10811
образование      4013
свадьба          2324
Name: purposes_list, dtype: int64

**Вывод**

Приведение целей получения кредита к общим категориям позволило выявить самые популярные цели,а именно около 50% кредитов на недвижимость, после по популярноти-кредиты на автомобиль и получение образования и на последнем месте - кредиты на свадьбу. Лемматизация позволила упростить категоризацию цели кредита для оценки гипотез. 

### Категоризация данных

Сгруппируем данные по типу 'семейное положение', и посчитаем количество событий. 

In [49]:
df['family_status'].value_counts().sort_index() 

в разводе                 1195
вдовец / вдова             959
гражданский брак          4151
женат / замужем          12339
не женат / не замужем     2810
Name: family_status, dtype: int64

Из данных видно,что более 50% женат/замужем

Сгруппируем данные по типу 'количество детей в семье', и посчитаем количество событий. 

In [50]:
df['children'].value_counts().sort_index()

0    14138
1     4808
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Из данных видно,что более 65% не имеют детей

Сгруппируем данные по типу 'цель получения кредита', и посчитаем количество событий. 

In [51]:
df['purposes_list'].value_counts().sort_index()

автомобиль       4306
недвижимость    10811
образование      4013
свадьба          2324
Name: purposes_list, dtype: int64

Категории позволят упростить подтвержение или опровержение гипотезы.Добавим категории по возрасту и доходу

In [52]:
def age_group(age):
    if age < 31 :
        return 'молодые'
    if 31 <= age <= 60:
        return 'средние'
    if age > 60:
        return 'пенсионеры'
    return 'группа не определена'
df['age_group'] = df['dob_years'].apply(age_group)


In [53]:
df['age_group'].value_counts().sort_index()

молодые        3717
пенсионеры     2126
средние       15611
Name: age_group, dtype: int64

Из данных видно,что более 70% в возрасте от 31 до 60 лет.

In [54]:
df['total_income'].value_counts()

142594    1070
172357     502
118514     387
150447     145
144533       3
          ... 
179138       1
61724        1
99269        1
151371       1
126820       1
Name: total_income, Length: 18608, dtype: int64

In [55]:
def income_group(income):
    if income <= 120000 :
        return 'низкий доход'
    if 120000 < income <= 200000:
        return 'средний доход'
    if income > 200000:
        return 'высокий доход'
    return 'группа не определена'
df['income_group'] = df['total_income'].apply(income_group)


In [56]:
df['income_group'].value_counts().sort_index()

высокий доход    5067
низкий доход     7231
средний доход    9156
Name: income_group, dtype: int64

Из данных видно,что боле 40 % зарабатывают более от 120000 до 200000 в месяц.

**Вывод**

Из категоризации данных можно сделать вывод, что более 70%  заемщиков имеют средний возраст от 31 до 60 лет. Также с средним доходом составляют более 40% от общего количества, более 50% замужем/женаты и более 65% не имеют детей.  Это говорит о том, что кредиты берет платежеспособная часть населения с высоким доходом, состоящие в официальном браке и не имеющие детей.

## Ответ на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [57]:
table_1 = pd.pivot_table(df, index='children', values='debt', aggfunc='count')
table_2 = pd.pivot_table(df, index='children', values='debt', aggfunc='sum')
children= table_1
children['non_returned_debt'] = table_2['debt']

children['returned_debt'] = children['debt'] - children['non_returned_debt']
children['return_ratio'] = children['returned_debt'] / children['debt']
children.sort_index()

,debt,non_returned_debt,returned_debt,return_ratio
children,,,,
0,14138,1064,13074,0.924742
1,4808,444,4364,0.907654
2,2128,202,1926,0.905075
3,330,27,303,0.918182
4,41,4,37,0.902439
5,9,0,9,1.000000


 Есть зависимость между наличием детей и возвратом кредита в срок.

 1. Самый высокий процент показали "5 детей" - 100%
 2. Следом категория "без детей" - 92,50%
 3. Далее "3 ребенка" - 91,81%
 4. "1 ребенок" - 90,78%
 5. И "4 ребенка" - 90,24%

**Вывод**

Из выборки мы видем,что заемщики без детей реже других допускают просрочки по кредиту, заемщики с 4 детьми чаще всего задерживают оплату,заемщики с 5 детьми не задерживают оплату,но на этот показатель можно не обращать внимания,т.к. всего 9 человек из наших данных имеют такое количество детей. Если сравнивать наличие детей и полное их отстутсвие и возврат кредита в срок, то можно утверждать,что на вовзрат кредита наличие детей влияет, все категории граждан задерживают оплату с почти одинаковой частотой.

**- Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [58]:
table_1 = pd.pivot_table(df, index='family_status', values='debt', aggfunc='count')
table_2 = pd.pivot_table(df, index='family_status', values='debt', aggfunc='sum')
family= table_1
family['non_returned_debt'] = table_2['debt']

family['returned_debt'] = family['debt'] - family['non_returned_debt']
family['return_ratio'] = family['returned_debt'] / family['debt']
family

,debt,non_returned_debt,returned_debt,return_ratio
family_status,,,,
в разводе,1195,85,1110,0.928870
вдовец / вдова,959,63,896,0.934307
гражданский брак,4151,388,3763,0.906529
женат / замужем,12339,931,11408,0.924548
не женат / не замужем,2810,274,2536,0.902491


Есть зависимость между семейным положением и возвратом кредита в срок.

1. Самый высокий процент показали "вдовец/вдова" - 93,43%
2. Следом категория "в разводе" - 92,88%
3. Далее "женат / замужем" - 92,47%
4. "Гражданский брак" - 90,7%
5. И "Не женат / не замужем" - 90,25%

**Вывод**


Из выборки мы видем,что чаще всего допускают задержку кредита не женатые/не замужние,либо состоящие в гражданском браке,реже всего вдовцы/вдовы.Можно сделать вывод, что семейные и либо те, кто бывал в браке, более надежные.

**- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [59]:
table_1 = pd.pivot_table(df, index='income_group', values='debt', aggfunc='count')
table_2 = pd.pivot_table(df, index='income_group', values='debt', aggfunc='sum')
income= table_1
income['non_returned_debt'] = table_2['debt']
income['returned_debt'] = income['debt'] - income['non_returned_debt']
income['return_ratio'] = income['returned_debt'] / income['debt']
income.sort_index()

,debt,non_returned_debt,returned_debt,return_ratio
income_group,,,,
высокий доход,5067,358,4709,0.929347
низкий доход,7231,586,6645,0.918960
средний доход,9156,797,8359,0.912953


Есть зависимость между уровнем дохода и возвратом кредита в срок.

  1. На первом месте уровень дохода от 200000 - 92,93%
  2. На втором уровень до 120000 - 91,92%
  3. Потом доход от 120000 до 200000 - 91,33%

**Вывод**

 Чаще всего задерживают кредиты заемщики с средним уровнем дохода, но не значительно чаще,чем с уровнем дохода ниже.
Из этого следует сделать вывод, что уровень дохода влияет на возврат кредита.


**- Как разные цели кредита влияют на его возврат в срок?**

In [60]:
table_1 = pd.pivot_table(df, index='purposes_list', values='debt', aggfunc='count')
table_2 = pd.pivot_table(df, index='purposes_list', values='debt', aggfunc='sum')
purposes_list = table_1
purposes_list['non_returned_debt'] = table_2['debt']
purposes_list['returned_debt'] = purposes_list['debt'] - purposes_list['non_returned_debt']
purposes_list['return_ratio'] = purposes_list['returned_debt'] / purposes_list['debt']
purposes_list.sort_index()

,debt,non_returned_debt,returned_debt,return_ratio
purposes_list,,,,
автомобиль,4306,403,3903,0.906410
недвижимость,10811,782,10029,0.927666
образование,4013,370,3643,0.907800
свадьба,2324,186,2138,0.919966


Имеется зависимость между целью и возвратом кредита в срок.

 1. Самый популярный тип кредита и самый возвращаемый - недвижимость - 92,78%
 2. За ней свадьба - 92,07%
 3. Образование - 90,80%
 4. Автомобиль - 90,66%
 
 **Вывод**
 
 Реже всего возвращют вовемя кредит за автомобиль, чаще всего - за недвижимость.

##  Общий вывод

Определим есть ли зависимость между типом занятости и возвратом кредита в срок?

In [61]:

table_1 = pd.pivot_table(df, index='income_type', values='debt', aggfunc='count')
table_2 = pd.pivot_table(df, index='income_type', values='debt', aggfunc='sum')
income_type = table_1
income_type['non_returned_debt'] = table_2['debt']
income_type['returned_debt'] = income_type['debt'] - income_type['non_returned_debt']
income_type['return_ratio'] = income_type['returned_debt'] / income_type['debt']
income_type.sort_index()

,debt,non_returned_debt,returned_debt,return_ratio
income_type,,,,
безработный,2,1,1,0.500000
в декрете,1,1,0,0.000000
госслужащий,1457,86,1371,0.940975
компаньон,5078,376,4702,0.925955
пенсионер,3829,216,3613,0.943588
предприниматель,2,0,2,1.000000
сотрудник,11084,1061,10023,0.904276
студент,1,0,1,1.000000


Имеется зависимость между типом занятости и возвратом кредита в срок.

 1. Чаще всего вовращают - преприниматели и студенты-100 %
 2. За ними - госслужащие - 94,1 %
 3. Пенсионеры - 94,39%
 4. Компаньоны - 92,60%
 5. Сотрудник - 90,45%
 6. Безработный - 50 %
 7. В декрете - 0 %
 
 
 **Вывод**
 
 Из данных можно сделать вывод, что чаще всего возращают кредит - госслужащие, а реже - сотрудники,т.к. в декрете, бузработных, предпринимателй и студентов мало в выборке (всего 6 человек).


Определим есть ли зависимость между уровнем образования и возвратом кредита в срок ?

In [62]:
table_1 = pd.pivot_table(df, index='education', values='debt', aggfunc='count')
table_2 = pd.pivot_table(df, index='education', values='debt', aggfunc='sum')
education = table_1
education['non_returned_debt'] = table_2['debt']
education['returned_debt'] = education['debt'] - education['non_returned_debt']
education['return_ratio'] = education['returned_debt'] / education['debt']
education.sort_index()

,debt,non_returned_debt,returned_debt,return_ratio
education,,,,
высшее,5250,278,4972,0.947048
начальное,282,31,251,0.890071
неоконченное высшее,744,68,676,0.908602
среднее,15172,1364,13808,0.910098
ученая степень,6,0,6,1.000000


Имеется зависимость между уровнем образования и возвратом кредита в срок.

 1. Чаще всего вовращают - с ученой степенью-100 %
 2. За ними - с высшим образованием - 94,71 %
 3. с средним - 91,04%
 4. С неконченным высшим - 90,86%
 5. С начальным - 89%

 
 
 **Вывод**
 
 Из данных можно сделать вывод, что чаще всего возращают кредит люди с ученой степенью,а реже с начальным образованием.


Определим есть ли зависимость между возрастом и возвратом кредита в срок ?

In [63]:
table_1 = pd.pivot_table(df, index='age_group', values='debt', aggfunc='count')
table_2 = pd.pivot_table(df, index='age_group', values='debt', aggfunc='sum')
age_group = table_1
age_group['non_returned_debt'] = table_2['debt']
age_group['returned_debt'] = age_group['debt'] - age_group['non_returned_debt']
age_group['return_ratio'] = age_group['returned_debt'] / age_group['debt']
age_group.sort_index()

,debt,non_returned_debt,returned_debt,return_ratio
age_group,,,,
молодые,3717,403,3314,0.891579
пенсионеры,2126,101,2025,0.952493
средние,15611,1237,14374,0.920761


Имеется зависимость между возрастом и возвратом кредита в срок.

 1. Чаще всего вовращают - пенсионеры -95,28 %
 2. За ними -средние - 92,10 %
 3. Реже всего- молодые - 89,17%


 
 
 **Вывод**
 
 Из данных можно сделать вывод, что чаще всего возращают пенсионеры и реже - молодые до 30 лет.


Подводим итоги и делаем общий вывод.

На основе всех вышеприведенных таблиц выявляем закономерности, а именно:
- Чем выше образование, тем ответственнее понимание по возращнию кредита
- Чем выше возраст, тем чаще возврат кредит и более ответственность
- Чем выше дожность, чем больше возращаемость кредита
- Высокий доход положительно влияет на возврат кредита
- Наличие детей отрицательно влиятет на возрат кредита
- Реже всего возрат кредита на автомобиль, чем на недвижимость
- Семейные люди, либо те, кто был в браке чаще возращают кредит

**Опишем положительного и отрицательного кандидата на получение кредита**

Положительный:
- Человек, который стал вдовцом/вдовой, который не имеет детей, с высшим образованием, доходом от 200000, преобретающий недвижимость.

Отрицательный:
- Человек, который не женат/не замужем, с начальным образованием, с доходом до 120000, покупающий автомобиль.